In [1]:
from multiprocessing import cpu_count

ncpu = cpu_count()
print("{0} CPUs".format(ncpu))

4 CPUs


In [2]:
# import of libraries we will use
%matplotlib inline
import random
#import tsfresh
import os
import math
from scipy import stats
from scipy.spatial.distance import pdist
from math import sqrt, log, floor
from fastdtw import fastdtw
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error

#Ajustes gráficos
sns.set(style='white')
# "Fijar" randomnes para reproducibilidad


In [3]:
path = "./data/Notificaciones/"
filename_read = os.path.join(path,"normalizado.csv")
dataset1 = pd.read_csv(filename_read,delimiter=",",engine='python')
dataset1

,Unnamed: 0,anio,casos,distrito_nombre,distrito_notif,incidencia,semana,0
0,0,2009.0,0.0,LOMA PLATA,0,0.0,1.0,NaN
1,1,2009.0,0.0,LOMA PLATA,0,0.0,2.0,NaN
2,2,2009.0,0.0,LOMA PLATA,0,0.0,3.0,NaN
3,3,2009.0,0.0,LOMA PLATA,0,0.0,4.0,NaN
4,4,2009.0,0.0,LOMA PLATA,0,0.0,5.0,NaN
...,...,...,...,...,...,...,...,...
57500,48,2013.0,0.0,JUAN EULOGIO ESTIGARRIBIA,0,0.0,49.0,NaN
57501,49,2013.0,0.0,JUAN EULOGIO ESTIGARRIBIA,0,0.0,50.0,NaN
57502,50,2013.0,0.0,JUAN EULOGIO ESTIGARRIBIA,0,0.0,51.0,NaN
57503,51,2013.0,0.0,JUAN EULOGIO ESTIGARRIBIA,0,0.0,52.0,NaN


In [4]:
dataset1 = dataset1.replace([np.inf, -np.inf], np.nan).fillna(0.000001)

In [5]:
#check the dtypes
dataset1.dtypes

Unnamed: 0           int64
anio               float64
casos              float64
distrito_nombre     object
distrito_notif      object
incidencia         float64
semana             float64
0                  float64
dtype: object

In [6]:
dataframe=dataset1[['incidencia']]
dataset = dataframe.values
dataset = dataset.astype('float32')

In [7]:
#creating the train and validation set
train = dataset[:int(0.6*(len(dataset)))]
test = dataset[int(0.6*(len(dataset))):]
print(len(train), len(test))

34503 23002


In [8]:
lista=['SAN LORENZO','CAPIATA','CAAGUAZU','AREGUA','SALTO DEL GUAIRA','CHORE','JUAN LEON MALLORQUIN','SANTA ROSA DEL AGUARAY','QUIINDY','EUSEBIO AYALA','ENCARNACION','SAN PEDRO DEL YCUAMANDIYU','CAPITAN MIRANDA','YHU','SANTA RITA']

In [9]:
def mape(A, F):
    return (1/len(A) * np.sum(np.abs(F - A) / (np.abs(A) +1)))*100
    #return np.mean(np.abs((A - F) / A)) * 100

RANDOM FOREST

In [10]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [12]:
from sklearn.ensemble._forest import RandomForestRegressor
def distRF(dataset1):
    dataframe=dataset1[['casos']]
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset)-15,:]
    print(len(train), len(test))
    # build our RF model
    RF_Model = RandomForestRegressor(n_estimators=1000,max_features=1, oob_score=True)

    # let's get the labels and features in order to run our 
    # model fitting
    #labels = y_train#[:, None]
    #features = X_train[:, None]
    #features, labels = create_dataset(train, look_back)
    look_back=1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    # Fit the RF model with features and labels.
    rgr=RF_Model.fit(trainX, trainY)

    # Now that we've run our models and fit it, let's create
    # dataframes to look at the results
    testPredict = RF_Model.predict(testX)
    #print(len(testPredict))
    #print(len(testY))
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    stestScore = mape(testY,testPredict)
    print('Test Score: %.4f RMSE' % (testScore))
    #real=plt.plot(test, label="Real")
    #predict=plt.plot(testPredict, label="Predict")
    #plt.show()
    return testScore,stestScore

In [13]:
from sklearn.linear_model import Lasso
def distLASSO(dataset1):
    dataframe=dataset1[['incidencia']]
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset)-15,:]
    look_back=1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    print(len(train), len(test))
    model_lasso = Lasso(alpha=0.00001)
    model_lasso.fit(trainX, trainY) 
    #pred_train_lasso= model_lasso.predict(X_train)
    #print(np.sqrt(mean_squared_error(y_train,pred_train_lasso)))
    #print(r2_score(y_train, pred_train_lasso))

    pred_test_lasso= model_lasso.predict(testX)
    testScore=(np.sqrt(mean_squared_error(testY,pred_test_lasso)))
    stestScore = mape(testY,pred_test_lasso)
    #print(r2_score(y_test, pred_test_lasso))
    print('Test Score: %.4f RMSE' % (testScore))
    #real=plt.plot(test, label="Real")
    #predict=plt.plot(pred_test_lasso, label="Predict")
    #plt.show()
    return testScore, stestScore

In [14]:
from sklearn.svm import SVR
def distSVR(dataset1):
    dataframe=dataset1[['incidencia']]
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset)-15,:]
    print(len(train), len(test))
    look_back=1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    
    #clf = SVR()
    #clf = SVR(C=8.5, epsilon=0.2)
    clf = SVR(C=2037, epsilon=0.98)
    clf.fit(trainX, trainY)
    pred=clf.predict(testX) 
    testScore=(np.sqrt(mean_squared_error(testY,pred))) 
    stestScore = mape(testY,pred)
    #print("Largo ",len(testX))
    #print('Test Score: %.4f RMSE' % (testScore))
    #real=plt.plot(test, label="Real")
    #predict=plt.plot(pred, label="Predict")
    #plt.show()
    return testScore,stestScore

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM
def distLSTM(dataset):
    #np.random.seed(7)
    # Arreglar el dataset
    dataframe=dataset[['incidencia']]
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    # split into train and test sets
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    look_back = 1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY,validation_data=(testX, testY), epochs=100, batch_size=1, verbose=0)
    train_mse = model.evaluate(trainX, trainY, verbose=0)
    test_mse = model.evaluate(testX, testY, verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_mse, test_mse))
    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    stestScore = mape(testY[0], testPredict[:,0])
    return testScore, stestScore

In [16]:
from keras.layers import GRU
def distGRU(dataset):
    #np.random.seed(7)
    # Arreglar el dataset
    dataframe=dataset[['incidencia']]
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    # split into train and test sets
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    look_back = 1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    model = Sequential()
    model.add(GRU(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY,validation_data=(testX, testY), epochs=100, batch_size=1, verbose=0)
    train_mse = model.evaluate(trainX, trainY, verbose=0)
    test_mse = model.evaluate(testX, testY, verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_mse, test_mse))
    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    
    stestScore = mape(testY[0], testPredict[:,0])
    return testScore, stestScore

In [17]:
from keras.layers import SimpleRNN
def distRNN(dataset):
    #np.random.seed(7)
    # Arreglar el dataset
    dataframe=dataset[['incidencia']]
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    # split into train and test sets
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    look_back = 1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    model = Sequential()
    model.add(SimpleRNN(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY,validation_data=(testX, testY), epochs=100, batch_size=1, verbose=0)
    train_mse = model.evaluate(trainX, trainY, verbose=0)
    test_mse = model.evaluate(testX, testY, verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_mse, test_mse))
    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    stestScore = mape(testY[0], testPredict[:,0])
    return testScore, stestScore

In [18]:
lista=['SAN LORENZO','CAPIATA','CAAGUAZU','AREGUA','SALTO DEL GUAIRA','CHORE','JUAN LEON MALLORQUIN','SANTA ROSA DEL AGUARAY','QUIINDY','EUSEBIO AYALA','ENCARNACION','SAN PEDRO DEL YCUAMANDIYU','CAPITAN MIRANDA','YHU','SANTA RITA']
print(lista)
rmse_RF = []
rmse_SVR = []
rmse_LASSO = []
rmse_LSTM = []
rmse_GRU = []
rmse_RNN = []
mape_RF = []
mape_SVR = []
mape_LASSO = []
mape_LSTM = []
mape_GRU = []
mape_RNN = []
for dist in lista:
    #print(C)
    departamento=dataset1['distrito_nombre']==dist
    datasetxdist=dataset1[departamento]
    print("Distrito: ",dist)
    print("Longitud del dataset: ",len(datasetxdist))
    #distrito=dataset['distrito_nombre']==dist
    #datasetxdist=dataset[distrito]
    #trainScore,testScore,reg=distLSTM(datasetxdist)
    rmseRF,mapeRF = distRF(datasetxdist)
    rmseSVR,mapeSVR = distSVR(datasetxdist)
    rmseLASSO,mapeLASSO = distLASSO(datasetxdist)
    rmseGRU,mapeGRU = distGRU(datasetxdist)
    rmseRNN,mapeRNN = distRNN(datasetxdist)
    rmseLSTM,mapeLSTM = distLSTM(datasetxdist)
    #print("trainScore",trainScore)
    #print("testScore",testScore)
    #print("reg",reg)
    #Para el dataframe
    rmse_SVR.append(rmseSVR)
    rmse_RF.append(rmseRF)
    rmse_LASSO.append(rmseLASSO)
    rmse_LSTM.append(rmseLSTM)
    rmse_GRU.append(rmseGRU)
    rmse_RNN.append(rmseRNN)
    mape_SVR.append(mapeSVR)
    mape_RF.append(mapeRF)
    mape_LASSO.append(mapeLASSO)
    mape_LSTM.append(mapeLSTM)
    mape_GRU.append(mapeGRU)
    mape_RNN.append(mapeRNN)
    #MSETest.append(testScore)
    #Registros.append(reg)
    data_tuples = list(zip(lista,rmse_SVR,rmse_RF,rmse_LASSO,rmse_LSTM,rmse_GRU,rmse_RNN))
    TablaRMSE = pd.DataFrame(data_tuples, columns =['Distrito', 'SVR','RF','LASSO','LSTM','GRU','RNN']) 
    data_tuples = list(zip(lista,mape_SVR,mape_RF,mape_LASSO,mape_LSTM,mape_GRU,mape_RNN))
    TablaMAPE = pd.DataFrame(data_tuples, columns =['Distrito', 'SVR','RF','LASSO','LSTM','GRU','RNN']) 
    data_tuples = list(zip(lista,rmse_SVR,mape_SVR,rmse_RF,mape_RF,rmse_LASSO,mape_LASSO,rmse_LSTM,mape_LSTM,rmse_GRU,mape_GRU,rmse_RNN,mape_RNN))
    Tabla = pd.DataFrame(data_tuples, columns =['Distrito','SVR_r','SVR_m','RF_r','RF_m','LASSO_r','LASSO_m','LSTM_r','LSTM_m','GRU_r','GRU_m','RNN_r','RNN_m']) 

['SAN LORENZO', 'CAPIATA', 'CAAGUAZU', 'AREGUA', 'SALTO DEL GUAIRA', 'CHORE', 'JUAN LEON MALLORQUIN', 'SANTA ROSA DEL AGUARAY', 'QUIINDY', 'EUSEBIO AYALA', 'ENCARNACION', 'SAN PEDRO DEL YCUAMANDIYU', 'CAPITAN MIRANDA', 'YHU', 'SANTA RITA']
Distrito:  SAN LORENZO
Longitud del dataset:  265
177 73
Test Score: 255.8329 RMSE
177 73
177 73
Test Score: 0.0402 RMSE
Train: 0.000, Test: 0.014
Train: 0.000, Test: 0.010
Train: 0.000, Test: 0.009
Distrito:  CAPIATA
Longitud del dataset:  265
177 73
Test Score: 341.9908 RMSE
177 73
177 73
Test Score: 0.0701 RMSE
Train: 0.000, Test: 0.009
Train: 0.000, Test: 0.008
Train: 0.000, Test: 0.009
Distrito:  CAAGUAZU
Longitud del dataset:  265
177 73
Test Score: 33.0365 RMSE
177 73
177 73
Test Score: 0.0282 RMSE
Train: 0.000, Test: 0.014
Train: 0.000, Test: 0.019
Train: 0.000, Test: 0.020
Distrito:  AREGUA
Longitud del dataset:  265
177 73
Test Score: 107.6624 RMSE
177 73
177 73
Test Score: 0.0741 RMSE
Train: 0.000, Test: 0.008
Train: 0.000, Test: 0.007
Tra

In [19]:
    data_tuples = list(zip(lista,rmse_SVR,mape_SVR,rmse_RF,mape_RF,rmse_LASSO,mape_LASSO,rmse_LSTM,mape_LSTM,rmse_GRU,mape_GRU,rmse_RNN,mape_RNN))
    Tabla = pd.DataFrame(data_tuples, columns =['Distrito','SVR_r','SVR_m','RF_r','RF_m','LASSO_r','LASSO_m','LSTM_r','LSTM_m','GRU_r','GRU_m','RNN_r','RNN_m'])

In [20]:
Tabla

,Distrito,SVR_r,SVR_m,RF_r,RF_m,LASSO_r,LASSO_m,LSTM_r,LSTM_m,GRU_r,GRU_m,RNN_r,RNN_m
0,SAN LORENZO,0.106971,5.936013,255.832874,125.579317,0.040213,1.784389,0.034098,1.456785,0.042651,1.714074,0.035019,1.491714
1,CAPIATA,0.167465,7.938011,341.990765,127.171730,0.070130,2.234351,0.068480,1.830307,0.066894,1.832259,0.063428,1.841415
2,CAAGUAZU,0.027186,1.270002,33.036504,54.095954,0.028192,1.243725,0.015467,0.586246,0.012877,0.489897,0.015055,0.558080
3,AREGUA,0.162959,7.374440,107.662352,64.322125,0.074148,2.403026,0.065899,1.934264,0.064633,1.945557,0.063654,2.078647
4,SALTO DEL GUAIRA,0.032327,2.909264,6.540395,40.612915,0.018474,0.711352,0.013663,0.495130,0.013471,0.504963,0.013163,0.484270
5,CHORE,0.019178,1.884179,1.516136,52.067429,0.005105,0.256645,0.004471,0.247298,0.004456,0.233623,0.004582,0.225625
6,JUAN LEON MALLORQUIN,0.009535,0.748916,1.647557,12.292464,0.008785,0.276763,0.006310,0.194027,0.006436,0.218630,0.006647,0.166728
7,SANTA ROSA DEL AGUARAY,0.010798,1.003250,2.780619,37.986380,0.008055,0.321041,0.007433,0.264415,0.007374,0.262883,0.007181,0.255769
8,QUIINDY,0.025693,2.496413,1.727200,53.636067,0.006508,0.369571,0.006355,0.385565,0.006189,0.329167,0.006142,0.332289
9,EUSEBIO AYALA,0.016106,1.527313,1.721174,26.230293,0.008028,0.362884,0.006897,0.254955,0.006864,0.259475,0.006768,0.289234


In [21]:
#-------------------------------------

In [22]:
def highlight_min(s):
    '''
    highlight the maximum in a Series green.
    '''
    is_min = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_min]

In [23]:
Tabla.style.apply(highlight_min,subset=pd.IndexSlice[0:14,['SVR_r','SVR_m','RF_r','RF_m','LASSO_r','LASSO_m','LSTM_r','LSTM_m','GRU_r','GRU_m','RNN_r','RNN_m']],axis=1)

,Distrito,SVR_r,SVR_m,RF_r,RF_m,LASSO_r,LASSO_m,LSTM_r,LSTM_m,GRU_r,GRU_m,RNN_r,RNN_m
0,SAN LORENZO,0.106971,5.936013,255.832874,125.579317,0.040213,1.784389,0.034098,1.456785,0.042651,1.714074,0.035019,1.491714
1,CAPIATA,0.167465,7.938011,341.990765,127.171730,0.070130,2.234351,0.068480,1.830307,0.066894,1.832259,0.063428,1.841415
2,CAAGUAZU,0.027186,1.270002,33.036504,54.095954,0.028192,1.243725,0.015467,0.586246,0.012877,0.489897,0.015055,0.558080
3,AREGUA,0.162959,7.374440,107.662352,64.322125,0.074148,2.403026,0.065899,1.934264,0.064633,1.945557,0.063654,2.078647
4,SALTO DEL GUAIRA,0.032327,2.909264,6.540395,40.612915,0.018474,0.711352,0.013663,0.495130,0.013471,0.504963,0.013163,0.484270
5,CHORE,0.019178,1.884179,1.516136,52.067429,0.005105,0.256645,0.004471,0.247298,0.004456,0.233623,0.004582,0.225625
6,JUAN LEON MALLORQUIN,0.009535,0.748916,1.647557,12.292464,0.008785,0.276763,0.006310,0.194027,0.006436,0.218630,0.006647,0.166728
7,SANTA ROSA DEL AGUARAY,0.010798,1.003250,2.780619,37.986380,0.008055,0.321041,0.007433,0.264415,0.007374,0.262883,0.007181,0.255769
8,QUIINDY,0.025693,2.496413,1.727200,53.636067,0.006508,0.369571,0.006355,0.385565,0.006189,0.329167,0.006142,0.332289
9,EUSEBIO AYALA,0.016106,1.527313,1.721174,26.230293,0.008028,0.362884,0.006897,0.254955,0.006864,0.259475,0.006768,0.289234


In [24]:
TablaRMSE.style.apply(highlight_min,subset=pd.IndexSlice[0:14,['SVR','RF','LASSO','LSTM','GRU','RNN']],axis=1)

,Distrito,SVR,RF,LASSO,LSTM,GRU,RNN
0,SAN LORENZO,0.106971,255.832874,0.040213,0.034098,0.042651,0.035019
1,CAPIATA,0.167465,341.990765,0.070130,0.068480,0.066894,0.063428
2,CAAGUAZU,0.027186,33.036504,0.028192,0.015467,0.012877,0.015055
3,AREGUA,0.162959,107.662352,0.074148,0.065899,0.064633,0.063654
4,SALTO DEL GUAIRA,0.032327,6.540395,0.018474,0.013663,0.013471,0.013163
5,CHORE,0.019178,1.516136,0.005105,0.004471,0.004456,0.004582
6,JUAN LEON MALLORQUIN,0.009535,1.647557,0.008785,0.006310,0.006436,0.006647
7,SANTA ROSA DEL AGUARAY,0.010798,2.780619,0.008055,0.007433,0.007374,0.007181
8,QUIINDY,0.025693,1.727200,0.006508,0.006355,0.006189,0.006142
9,EUSEBIO AYALA,0.016106,1.721174,0.008028,0.006897,0.006864,0.006768


In [25]:
TablaMAPE.style.apply(highlight_min,subset=pd.IndexSlice[0:14,['SVR','RF','LASSO','LSTM','GRU','RNN']],axis=1)

,Distrito,SVR,RF,LASSO,LSTM,GRU,RNN
0,SAN LORENZO,5.936013,125.579317,1.784389,1.456785,1.714074,1.491714
1,CAPIATA,7.938011,127.171730,2.234351,1.830307,1.832259,1.841415
2,CAAGUAZU,1.270002,54.095954,1.243725,0.586246,0.489897,0.558080
3,AREGUA,7.374440,64.322125,2.403026,1.934264,1.945557,2.078647
4,SALTO DEL GUAIRA,2.909264,40.612915,0.711352,0.495130,0.504963,0.484270
5,CHORE,1.884179,52.067429,0.256645,0.247298,0.233623,0.225625
6,JUAN LEON MALLORQUIN,0.748916,12.292464,0.276763,0.194027,0.218630,0.166728
7,SANTA ROSA DEL AGUARAY,1.003250,37.986380,0.321041,0.264415,0.262883,0.255769
8,QUIINDY,2.496413,53.636067,0.369571,0.385565,0.329167,0.332289
9,EUSEBIO AYALA,1.527313,26.230293,0.362884,0.254955,0.259475,0.289234


In [ ]:
#Tabla.to_csv("./data/Resultados/SyntheticSIR/Benchmark.csv")

In [ ]:
-----------------------------------------------